In [ ]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [ ]:
df = pd.read_excel('yourdata.xlsx')
df.head()

In [ ]:
# Feats that always same value
a=df.columns[df.nunique() <= 1]
constant_feat=list(a)
df=df.drop(columns=constant_feat)
print(constant_feat)

In [ ]:
df.groupby(['diagnosis']).size()

In [ ]:
df1=df[(df["diagnosis"]==2)]
df2=df[(df["diagnosis"]==3)]
print(df1.shape)
print(df2.shape)
df=pd.concat((df1,df2))

In [ ]:
select_feat=['A1','A2', 'A3','A4','A5','A6','A7','A8','A9','B1','B2','B3','B4','B5_B6','B6_B7','B7_B9','B9_B11','B10_B12','C1','D1','D2','D3','D4','D5','E1','E2','E3','ADI_A50','ADI_A51','ADI_A57','ADI_A49','ADI_A62','ADI_A63','ADI_A64','ADI_A65','ADI_A52','ADI_A53','ADI_A54', 'ADI_A31','ADI_A55','ADI_A56','ADI_A58','ADI_A59','ADI_B42','ADI_B43','ADI_B44','ADI_B45','ADI_B47','ADI_B48','ADI_B61','ADI_B34','ADI_B35','ADI_B33','ADI_B36','ADI_B37','ADI_B38','ADI_C67','ADI_C68','ADI_C39','ADI_C70','ADI_C77','ADI_C78','ADI_C69','ADI_C71','ADI_D2','ADI_D9','ADI_D10','ADI_D86','ADI_D87']

In [ ]:
df=df[select_feat]
df=df.fillna(df.mean())
df=df.fillna(df.mean())
y=np.zeros((df.shape[0],))
x=df.to_numpy()
y[:df1.shape[0]]=0
y[df1.shape[0]:]=1
y=y.astype(int)
x = 2*((x - x.min(axis=0)) / (x.max(axis=0) - x.min(axis=0)))-1
print(x.shape)

In [ ]:
feat=df.columns.to_list()

In [ ]:
def my_svm_select_param(x,y):
    
    clf=SVC(probability=True)
    kf = KFold(n_splits=10,shuffle=True)
    a=np.arange(0,len(x))
    auc=0
    f1=0
    per=0
    rec=0
    acc=0
    conf=np.zeros((2,2))
    for train_index, test_index in kf.split(a):
    # define the models
        x_tr=x[train_index,:]
        y_tr=y[train_index]
        x_test=x[test_index,:]
        y_test=y[test_index]
        param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf','linear','poly']}
 
        grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
        grid.fit(x_tr, y_tr)
        y_pred = grid.predict(x_test)
        conf=conf+metrics.confusion_matrix(y_test,y_pred)
        f1=f1+metrics.f1_score(y_test,y_pred)
        rec=rec+metrics.recall_score(y_test,y_pred)
        per=per+metrics.precision_score(y_test,y_pred)
        acc=acc+metrics.accuracy_score(y_test,y_pred)
        #pred=grid.predict_proba(x_test,probability=True)
        #fpr, tpr, thresholds = metrics.roc_curve(y_test, pred[:,1])
        #auc=auc+metrics.auc(fpr, tpr)
        
        
        # print best parameter after tuning
        print(grid.best_params_)
 
        # print how our model looks after hyper-parameter tuning
        print(grid.best_estimator_)

    f1=f1/10
    rec=rec/10
    per=per/10
    acc=acc/10
    #auc=auc/10

    c=np.sum(conf,axis=1)
    a1=conf[0,0]/c[0]
    a2=conf[0,1]/c[0]
    a3=conf[1,0]/c[1]
    a4=conf[1,1]/c[1]
    conf_norm=np.array([[a1,a2],[a3,a4]])
    
    return (f1,rec,per,acc,conf_norm,y)

In [ ]:
(f1,rec,per,acc,conf_norm,y) =my_svm_select_param(x,y)
print('\n'+ 'f-score '+str(f1))
print('\n'+ 'recall '+str(rec))
print('\n'+'per ' +str(per))
print('\n'+ 'acc '+str(acc))
print('\n'+ 'confusion matrix '+'\n')
print(conf_norm)

In [ ]:
def my_svm(x,y):
    kf = KFold(n_splits=10,shuffle=True)
    a=np.arange(0,len(x))
    auc=0
    f1=0
    per=0
    rec=0
    acc=0
    conf=np.zeros((2,2))
    for train_index, test_index in kf.split(a):
    # define the models
        x_tr=x[train_index,:]
        y_tr=y[train_index]
        x_test=x[test_index,:]
        y_test=y[test_index]
        clf = svm.SVC(kernel='rbf',probability=True)
        clf.fit(x_tr, y_tr)
        y_pred = clf.predict(x_test)
        conf=conf+metrics.confusion_matrix(y_test,y_pred)
        f1=f1+metrics.f1_score(y_test,y_pred)
        rec=rec+metrics.recall_score(y_test,y_pred)
        per=per+metrics.precision_score(y_test,y_pred)
        acc=acc+metrics.accuracy_score(y_test,y_pred)
        pred=clf.predict_proba(x_test)
        fpr, tpr, thresholds = metrics.roc_curve(y_test, pred[:,1])
        auc=auc+metrics.auc(fpr, tpr)

    f1=f1/10
    rec=rec/10
    per=per/10
    acc=acc/10
    auc=auc/10

    c=np.sum(conf,axis=1)
    a1=conf[0,0]/c[0]
    a2=conf[0,1]/c[0]
    a3=conf[1,0]/c[1]
    a4=conf[1,1]/c[1]
    conf_norm=np.array([[a1,a2],[a3,a4]])
    
    return (f1,rec,per,acc,conf_norm,y,auc) 

In [ ]:
n_select=5
final_answer=np.zeros((n_select,3))

## First Feature

n_feat=x.shape[1]
acc=np.zeros((n_feat,1))
auc=np.zeros((n_feat,1))
for i in range(n_feat):
    
    d=x[:,i]
    d=np.reshape(d,(len(d),1))
    
    (f1,rec,per,acc1,conf_norm,y,auc1)=my_svm(d,y)
    acc[i,0]=acc1
    auc[i,0]=auc1
    
final_answer[0,0]=np.argmax(acc)
final_answer[0,1]=np.max(acc)
final_answer[0,2]=np.max(auc)
# Others featurs 
i=0
while (i<n_select-1):
    
    
    # Forward 
    
    acc=np.zeros((n_feat,1))
    
    for j in range(n_feat):
        
    
        if  not np.in1d(j,final_answer[:i+1,0]):
            
                
                index=final_answer[:i+1,0]
                index=np.append(index,j)
                index=index.astype(int)
                d=x[:,index]
                d=np.reshape(d,(len(d),i+2))
    
                (f1,rec,per,acc1,conf_norm,y,auc1)=my_svm(d,y)
                acc[j,0]=acc1
                auc[j,0]=auc1
    
    final_answer[i+1,0]=np.argmax(acc)
    final_answer[i+1,1]=np.max(acc)
    final_answer[i+1,2]=np.max(auc)
    feat_forward=np.argmax(acc)
    final_answer_back=final_answer[:i+1,0]
    final_answer_back=np.delete(final_answer_back,i)
    
    #Backward
    
    for j in range(n_feat):
        
        acc=np.zeros((n_feat,1))
        
        if  not np.in1d(j,final_answer[:,0]):
            
                
                index=final_answer_back
                index=np.append(index,j)
                index=index.astype(int)
                d=x[:,index]
                d=np.reshape(d,(len(d),i+1))
    
                (f1,rec,per,acc1,conf_norm,y,auc1)=my_svm(d,y)
                acc[j,0]=acc1
                auc[j,0]=auc1
            
        if final_answer[i,0]<np.max(acc):
            
            final_answer[i,0]=np.argmax(acc)
            final_answer[i,1]=np.max(acc)
            final_answer[i,2]=np.max(auc)
            
        
        
    
    #backwards
    
    i=i+1

In [ ]:
select=final_answer[:,0]
select=select.astype(int)

In [ ]:
for i in range(n_select):
    print('\n  '+select_feat[select[i]]+'       '+ str(final_answer[i,1]))

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats

# AUC comparison adapted from
# https://github.com/Netflix/vmaf/
def compute_midrank(x):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i + j - 1)
        i = j
    T2 = np.empty(N, dtype=np.float)
    # Note(kazeevn) +1 is due to Python using 0-based indexing
    # instead of 1-based in the AUC formula in the paper
    T2[J] = T + 1
    return T2


def compute_midrank_weight(x, sample_weight):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    cumulative_weight = np.cumsum(sample_weight[J])
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = cumulative_weight[i:j].mean()
        i = j
    T2 = np.empty(N, dtype=np.float)
    T2[J] = T
    return T2


def fastDeLong(predictions_sorted_transposed, label_1_count):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank(positive_examples[r, :])
        ty[r, :] = compute_midrank(negative_examples[r, :])
        tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
    v01 = (tz[:, :m] - tx[:, :]) / n
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def calc_pvalue(aucs, sigma):
    """Computes log(10) of p-values.
    Args:
       aucs: 1D array of AUCs
       sigma: AUC DeLong covariances
    Returns:
       log10(pvalue)
    """
    l = np.array([[1, -1]])
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
    return np.log10(2) + scipy.stats.norm.logsf(z, loc=0, scale=1) / np.log(10)


def compute_ground_truth_statistics(ground_truth, sample_weight=None):
    assert np.array_equal(np.unique(ground_truth), [0, 1])
    order = (-ground_truth).argsort()
    label_1_count = int(ground_truth.sum())
    if sample_weight is None:
        ordered_sample_weight = None
    else:
        ordered_sample_weight = sample_weight[order]

    return order, label_1_count, ordered_sample_weight


def delong_roc_variance(ground_truth, predictions):
    """
    Computes ROC AUC variance for a single set of predictions
    Args:
       ground_truth: np.array of 0 and 1
       predictions: np.array of floats of the probability of being class 1
    """
    sample_weight = None
    order, label_1_count, ordered_sample_weight = compute_ground_truth_statistics(
        ground_truth, sample_weight)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count, ordered_sample_weight)
    assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
    return aucs[0], delongcov


def delong_roc_test(ground_truth, predictions_one, predictions_two):
    """
    Computes log(p-value) for hypothesis that two ROC AUCs are different
    Args:
       ground_truth: np.array of 0 and 1
       predictions_one: predictions of the first model,
          np.array of floats of the probability of being class 1
       predictions_two: predictions of the second model,
          np.array of floats of the probability of being class 1
    """
    sample_weight = None
    order, label_1_count,ordered_sample_weight = compute_ground_truth_statistics(ground_truth)
    predictions_sorted_transposed = np.vstack((predictions_one, predictions_two))[:, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
    return calc_pvalue(aucs, delongcov)

In [ ]:
def my_svm_delog(x1,x2,y):
    kf = KFold(n_splits=10,shuffle=True)
    a=np.arange(0,len(x))
    y_pred1_1=np.zeros((1,))
    y_pred2_2=np.zeros((1,))
    y_truth=np.zeros((1,))
    for train_index, test_index in kf.split(a):
    # define the models
        x_tr1=x1[train_index,:]
        x_tr2=x2[train_index,:]

        
        x_test1=x1[test_index,:]
        x_test2=x2[test_index,:]
 
        
        y_test=y[test_index]
        y_tr=y[train_index]
        
        clf1 = svm.SVC(kernel='linear',probability=True)
        clf1.fit(x_tr1, y_tr)
        clf2 = svm.SVC(kernel='linear',probability=True)
        clf2.fit(x_tr2, y_tr)

        
        y_pred1 = clf1.predict_proba(x_test1)
        y_pred2 = clf2.predict_proba(x_test2)

        y_pred1_1=np.concatenate((y_pred1_1,y_pred1[:,1]))
        y_pred2_2=np.concatenate((y_pred2_2,y_pred2[:,1]))

        y_truth=np.concatenate((y_truth,y_test))
        
    y1=y_pred1_1[1:]
    y2=y_pred2_2[1:]

    y_t=y_truth[1:]
        
    return (y1,y2,y_t)

In [ ]:
y1,y2,y_t=my_svm_delog(x,x[:,select],y)

t1=delong_roc_test(y_t,y1,y2)
t1=10**(t1[0][0])



fpr, tpr, thresholds = metrics.roc_curve(y_t ,y1)
auc1=metrics.auc(fpr, tpr)
rec1=metrics.recall_score(y_t,y1.round())
per1=metrics.precision_score(y_t,y1.round())

fpr, tpr, thresholds = metrics.roc_curve(y_t, y2)
auc2=metrics.auc(fpr, tpr)
rec2=metrics.recall_score(y_t,y2.round())
per2=metrics.precision_score(y_t,y2.round())

In [ ]:
print('\n'+'p-vlue for best -features vs all features '+str(t1))

In [ ]:
print('\n'+'p-vlue for best -features vs all features '+str(t1))

print('\n'+'for all features AUC, Sensitivity and specificity {}  {} {}' .format(str(auc1),str(rec1),str(per1)))
print('\n'+'for selected features AUC, Sensitivity and specificity {}  {} {}' .format(str(auc2),str(rec2),str(per2)))